# 모든 종목 통합 저장

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from binance.client import Client
import requests
import time

# data.csv로 모든 종목 통합

class BinanceDataCollector:
    COLUMNS = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore']

    def __init__(self, api_key, api_secret):
        self.client = Client(api_key=api_key, api_secret=api_secret)
        self.hist_data = pd.DataFrame()

    def get_data(self, start_date, end_date, symbol):
        data = self.client.futures_historical_klines(
            symbol=symbol,
            interval='1m',
            start_str=start_date,
            end_str=end_date
        )
        df = pd.DataFrame(data, columns=self.COLUMNS)
        df['open_time'] = df.apply(lambda x: datetime.fromtimestamp(x['open_time'] // 1000), axis=1)
        df = df.drop(columns=['close_time', 'ignore'])
        df['symbol'] = symbol
        df.loc[:, 'open':'tb_quote_av'] = df.loc[:, 'open':'tb_quote_av'].astype(float)
        df['trades'] = df['trades'].astype(int)
        return df
    

class BinanceDataUpdater(BinanceDataCollector):
    def get_binance_server_time(self):
        url = 'https://fapi.binance.com/fapi/v1/time'
        response = requests.get(url)
        data = response.json()
        server_time = int(data['serverTime'])
        server_time = datetime.fromtimestamp(server_time / 1000)

        return server_time
          
    async def update_data(self, symbols):
        server_time = self.get_binance_server_time()
        next_minute = server_time.replace(second=0, microsecond=0) + timedelta(minutes=1)
        wait_time = (next_minute - server_time).total_seconds()
        
        print(wait_time)
        
        time.sleep(wait_time)

        hist_time_30 = next_minute - timedelta(minutes=30)
        # 시간을 문자열로 변환
        start_str = str(int(hist_time_30.timestamp() * 1000))
        end_str = str(int(next_minute.timestamp() * 1000))
        for symbol in symbols:
            df_hist = self.get_data(start_str, end_str, symbol)
            self.hist_data = pd.concat([self.hist_data, df_hist], ignore_index=True)
            
        self.hist_data.sort_values(by='open_time', inplace=True)
        
        while True:
            self.current_data = pd.DataFrame()
            server_time = self.get_binance_server_time()
            current_minute = server_time.replace(second=0, microsecond=0)
            next_minute = current_minute + timedelta(minutes=1)
            wait_time = (next_minute - server_time).total_seconds()
            
            print(f'Waiting for {wait_time} seconds.')
            time.sleep(wait_time)
            
            for symbol in symbols:
                start_str = str(int(current_minute.timestamp() * 1000))
                end_str = str(int(next_minute.timestamp() * 1000))
                df_real = self.get_data(start_str, end_str, symbol)
                self.current_data = pd.concat([self.current_data, df_real], ignore_index=True)
            
            self.hist_data = pd.concat([self.hist_data, self.current_data], ignore_index=True)
            with open('D:/csv/data.csv', 'a', newline='') as f:
                self.hist_data.to_csv(f, header=f.tell()==0, index=False)
            break
            

        
async def main():
    api_key = ''
    api_secret = ''
    symbols = ['BTCUSDT', 'ETHUSDT', 'DOGEUSDT']
    updater = BinanceDataUpdater(api_key, api_secret)
    await updater.update_data(symbols)

# 개별 csv 저장

In [ ]:
# 각각의 종목별 차트 데이터 수집


api_key = ''
api_secret = ''

from datetime import datetime
import pandas as pd

COLUMNS = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 
                   'tb_base_av', 'tb_quote_av', 'ignore']


from binance.client import Client
client = Client(api_key=api_key, api_secret=api_secret)

def get_data(start_date, end_date, symbol):
    data = []
    
    data = client.futures_historical_klines(
            symbol=symbol,
            interval='1m',
            start_str=start_date,
            end_str=end_date
        )
    
    # 전처리
    df = pd.DataFrame(data)
    df.columns = COLUMNS
    df['open_time'] = df.apply(lambda x:datetime.fromtimestamp(x['open_time'] // 1000), axis=1)
    df = df.drop(columns = ['close_time', 'ignore'])
    df['symbol'] = symbol
    df.loc[:, 'open':'tb_quote_av'] = df.loc[:, 'open':'tb_quote_av'].astype(float)  # string to float
    df['trades'] = df['trades'].astype(int)
    return df

start_date = '2024-01-14'
end_date = '2024-01-15'
symbols = [
    'BTCUSDT', 'ETHUSDT', 'SOLUSDT', 'TRBUSDT', 'LINKUSDT', 'BNBUSDT',
    'AVAXUSDT', 'MATICUSDT', 'XRPUSDT', 'ETCUSDT', 'FILUSDT', 'DOGEUSDT',
    'NEARUSDT', 'ADAUSDT', 'LTCUSDT', 'BCHUSDT', 'FTMUSDT', 'ATOMUSDT',
    'EOSUSDT', 'RUNEUSDT', 'CRVUSDT', 'UNIUSDT', 'TRXUSDT', 'MKRUSDT',
    'XTZUSDT', 'EGLDUSDT', 'ALGOUSDT', 'XMRUSDT', 'NEOUSDT', 'COMPUSDT',
    'THETAUSDT', 'SNXUSDT', 'XLMUSDT', 'ZILUSDT'
]
for i in symbols:
    df = get_data(start_date=start_date, end_date=end_date, symbol=i)
    df.to_csv(f'D:\csv\hist_coin_data\{i}.csv', index=False)